In [1]:
import pandas as pd
import re
import numpy as np

# PUNTO 1: For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [2]:
times_data=pd.read_csv("Downloads/2020/timesData.csv")
times_data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011


In [3]:
times_data['wrank_M']=times_data['world_rank'].str.extract(r'([0-9]*$)')
times_data['wrank_m']=times_data['world_rank'].str.extract(r'([0-9]+)')
times_data['wrank_M']=np.array(times_data['wrank_M'],dtype=int)
times_data['wrank_m']=np.array(times_data['wrank_m'],dtype=int)                                                           

In [4]:
mr_data=times_data.iloc[times_data.groupby('university_name')['year'].idxmax()]
mr_data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,wrank_M,wrank_m
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,800,601
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,250,201
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,300,251
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,106,106
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,350,301


In [5]:
lr_data=times_data.iloc[times_data.groupby('university_name')['year'].idxmin()]
lr_data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,wrank_M,wrank_m
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,800,601
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012,350,301
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012,350,301
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011,167,167
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012,300,276


# PUNTO 2:  For each university, compute the improvement in income between the least recent and the most recent data points

In [6]:
a=mr_data['income']
b=lr_data['income']
counts=[]
for (i,j) in list(zip(a,b)):
    if i=='-' or j=='-':
        counts.append(None)
    else:
        counts.append(float(i)-float(j))
dati={'university_name':mr_data['university_name'],'increase':counts}
pd.DataFrame(dati)

,university_name,increase
2405,AGH University of Science and Technology,NaN
2003,Aalborg University,7.3
2056,Aalto University,-0.3
1908,Aarhus University,6.8
2105,Aberystwyth University,-4.2
...,...,...
1856,École Normale Supérieure,6.4
2013,École Normale Supérieure de Lyon,5.6
1904,École Polytechnique,NaN
1833,École Polytechnique Fédérale de Lausanne,27.4


# PUNTO 3: Find the university with the largest increase computed in the previous point

In [7]:
pd.DataFrame(dati)[pd.DataFrame(dati)['increase']==pd.DataFrame(dati)['increase'].max()]['university_name']

1960    TU Dresden
Name: university_name, dtype: object

# PUNTO 4: For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

In [8]:
cwurData=pd.read_csv("Downloads/2020/cwurData.csv")
cwur_data=cwurData.iloc[cwurData.groupby('institution')['year'].idxmax()]

In [9]:
shanghaiData=pd.read_csv('Downloads/2020/shanghaiData.csv')
shanghaiData['wrank_M']=shanghaiData['world_rank'].str.extract(r'([0-9]*$)')
shanghaiData['wrank_m']=shanghaiData['world_rank'].str.extract(r'([0-9]+)')
shanghaiData['wrank_M']=np.array(shanghaiData['wrank_M'],dtype=int)
shanghaiData['wrank_m']=np.array(shanghaiData['wrank_m'],dtype=int)
shan_data=shanghaiData.iloc[shanghaiData.groupby('university_name')['year'].idxmax()]

In [10]:
fuso=pd.merge(shan_data,mr_data,on='university_name',how='outer')
fuso1=pd.merge(fuso,cwur_data,left_on='university_name',right_on='institution',how='outer')
fuso[['university_name','world_rank_x','world_rank_y']]
fuso1[['institution','world_rank','wrank_M_x','wrank_m_x','wrank_M_y','wrank_m_y']]
fuso2=pd.DataFrame(fuso1[['institution','world_rank','wrank_M_x','wrank_m_x','wrank_M_y','wrank_m_y']])
fuso2.head()

,institution,world_rank,wrank_M_x,wrank_m_x,wrank_M_y,wrank_m_y
0,Aalborg University,565.0,400.0,301.0,250.0,201.0
1,Aalto University,421.0,500.0,401.0,300.0,251.0
2,Aarhus University,122.0,73.0,73.0,106.0,106.0
3,NaN,NaN,150.0,101.0,NaN,NaN
4,Aix-Marseille University,206.0,150.0,102.0,300.0,251.0


In [73]:
fuso1[(fuso1['world_rank'].notnull())&(fuso1['wrank_M_x'].notnull())&(fuso1['wrank_M_y'].notnull())&(fuso1['wrank_m_x'].notnull())&(fuso1['wrank_m_y'].notnull())].shape

(362, 43)

In [74]:
fuso2['diff1']=abs(fuso2['world_rank']-fuso2['wrank_m_x'])
fuso2['diff2']=abs(fuso2['world_rank']-fuso2['wrank_m_y'])
fuso2['diff3']=abs(fuso2['wrank_M_x']-fuso2['wrank_m_y'])
fuso2['diff4']=abs(fuso2['wrank_M_y']-fuso2['wrank_m_x'])
fuso2['diff5']=abs(fuso2['wrank_m_x']-fuso2['wrank_M_x'])
fuso3=fuso2[['institution','diff1','diff2','diff3','diff4','diff5']]
def calcolaMaxDiff(row):
    return max(row[1],row[2],row[3],row[4],row[5])
fuso3.apply(calcolaMaxDiff,axis=1)
a1=fuso1['institution']
b1=fuso1['university_name']
total_universities=[]
for (i,j) in list(zip(a1,b1)):
    if pd.notnull(i):
        total_universities.append(i)
    if pd.isnull(i) and pd.notnull(j):
        total_universities.append(j)
diz={}
for (i,j) in zip(total_universities,fuso3.apply(calcolaMaxDiff,axis=1)):
    diz[i]=j
diz

{'Aalborg University': 364.0,
 'Aalto University': 249.0,
 'Aarhus University': 49.0,
 'Aix Marseille University': nan,
 'Aix-Marseille University': 198.0,
 'Aristotle University of Thessaloniki': 399.0,
 'Arizona State University': 96.0,
 'Arizona State University - Tempe': nan,
 'Auburn University': 199.0,
 'Autonomous University of Barcelona': 154.0,
 'Autonomous University of Madrid': 149.0,
 'Bangor University': 267.0,
 'Bar-Ilan University': 120.0,
 'Baylor College of Medicine': 49.0,
 'Beihang University': 404.0,
 'Beijing Normal University': 195.0,
 'Beijing University of Aeronautics and Astronautics': nan,
 'Ben-Gurion University of the Negev': 199.0,
 'Bielefeld University': 249.0,
 'Birkbeck, University of London': 497.0,
 'Boston College': 310.0,
 'Boston University': 9.0,
 'Brandeis University': 115.0,
 'Brigham Young University': 102.0,
 'Brown University': 32.0,
 'Brunel University': 99.0,
 'Cairo University': 399.0,
 'California Institute of Technology': 11.0,
 'Capital

# PUNTO 5: Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country

In [75]:
times_data['num_students1']=times_data['num_students'].str.extract(r'([0-9]*)')
times_data['num_students2']=times_data['num_students'].str.extract(r'([0-9]*$)')
times_data['f_m_ratio1']=times_data['female_male_ratio'].str.extract("(^[0-9]+)")
times_data['f_m_ratio2']=times_data['female_male_ratio'].str.extract(r'([0-9]+$)')
times_data.head()
times_data['num_students1']=np.array(times_data['num_students1'],dtype=float)
times_data['num_students2']=np.array(times_data['num_students2'],dtype=float)
times_data['f_m_ratio1']=np.array(times_data['f_m_ratio1'],dtype=float)
times_data['f_m_ratio2']=np.array(times_data['f_m_ratio2'],dtype=float)

In [76]:
times_data['num_students']=times_data['num_students1']*1000+times_data['num_students2']
times_data['female_male_ratio']=times_data['f_m_ratio1']/times_data['f_m_ratio2']
times_data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,wrank_M,wrank_m,num_students1,num_students2,f_m_ratio1,f_m_ratio2
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,20152.0,8.9,25%,NaN,2011,1,1,20.0,152.0,NaN,NaN
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,27%,0.492537,2011,2,2,2.0,243.0,33.0,67.0
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,11074.0,9.0,33%,0.587302,2011,3,3,11.0,74.0,37.0,63.0
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,15596.0,7.8,22%,0.724138,2011,4,4,15.0,596.0,42.0,58.0
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,7929.0,8.4,27%,0.818182,2011,5,5,7.0,929.0,45.0,55.0


In [77]:
MR_data=times_data.loc[times_data.groupby('university_name')['year'].idxmax()][['university_name','country','num_students','female_male_ratio','f_m_ratio1']]
MR_data['num_females']=MR_data['num_students']*(MR_data['f_m_ratio1']/100)
MR_data['num_males']=MR_data['num_students']-MR_data['num_females']
MR_data[['country','num_females','num_males']].groupby('country',as_index=False).sum()

,country,num_females,num_males
0,Argentina,67191.26,41181.74
1,Australia,391735.88,321640.12
2,Austria,68364.08,66112.92
3,Bangladesh,21323.44,41392.56
4,Belarus,20219.07,9083.93
...,...,...,...
67,Unisted States of America,0.00,0.00
68,United Arab Emirates,9516.44,4930.56
69,United Kingdom,711813.62,613028.38
70,United States of America,1601453.63,1556321.37


# PUNTO 6: Find the universities where the ratio between female and male is below the average ratio (computed over all universities)


In [78]:
MR_data[MR_data['f_m_ratio1']<=(MR_data['f_m_ratio1'].mean())][['university_name','num_females']]

,university_name,num_females
2003,Aalborg University,8362.56
2056,Aalto University,5151.68
2105,Aberystwyth University,4440.96
2406,Ajou University,4192.98
2408,Alexandria University,58618.26
...,...,...
2104,Zhejiang University,19478.28
1856,École Normale Supérieure,1104.00
2013,École Normale Supérieure de Lyon,1086.82
1904,École Polytechnique,437.22


# PUNTO 7:For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country) 

In [79]:
MR_data1=MR_data[MR_data['f_m_ratio1']<=(MR_data['f_m_ratio1'].mean())][['country','num_students']]
MR_data2=MR_data1.groupby('country',as_index=False)['num_students'].sum()
MR_data3=pd.merge(MR_data2,MR_data[['country','num_students']].groupby('country',as_index=False).sum(),on='country')
MR_data3['ratio_students']=MR_data3['num_students_x']/MR_data3['num_students_y']
MR_data3[['country','ratio_students']]

,country,ratio_students
0,Australia,0.146438
1,Austria,0.398633
2,Bangladesh,1.000000
3,Brazil,0.421244
4,Canada,0.077765
5,Chile,0.783885
6,China,0.623643
7,Colombia,0.288645
8,Czech Republic,0.190266
9,Denmark,0.382933


# PUNTO 8:  Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [80]:
edAtt_data=pd.read_csv("Downloads/2020/educational_attainment_supplementary_data.csv")
ed_att_data=(edAtt_data.dropna(subset=['country_name'])).dropna(subset=['series_name'])
ed_att_data.tail()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN
79049,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.14588,NaN,NaN


# PUNTO 9:  From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [81]:
lista_dataframe=[]
for i in ed_att_data.index:
    L1=[ed_att_data['country_name'][i] for _ in range(27)]
    L2=[ed_att_data['series_name'][i] for _ in range(27)]
    L3=list(ed_att_data.columns[2:29])
    L4=list(ed_att_data.iloc[i][2:29])
    lista_dataframe.append(pd.DataFrame(zip(L1,L2,L3,L4), columns=['country_name','series_name','year','value']))
pd.concat(lista_dataframe)

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33000
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1986,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1987,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1990,0.44000
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1991,NaN
...,...,...,...,...
22,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2010,NaN
23,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2011,NaN
24,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2012,9.14588
25,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2013,NaN


# PUNTO 10: For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking)

In [30]:
times_data['nome_università']=times_data['university_name']
rankTIMES=times_data.iloc[times_data.groupby('university_name')['year'].idxmax()][['nome_università','world_rank']]
shanghaiData['nome_università']=shanghaiData['university_name']
rankSHANGHAI=shanghaiData.iloc[shanghaiData.groupby('university_name')['year'].idxmax()][['nome_università','world_rank']]
cwurData['nome_università']=cwurData['institution']
rankCWUR=cwurData.iloc[cwurData.groupby('institution')['year'].idxmax()][['nome_università','world_rank']]
pd.concat([rankTIMES,rankSHANGHAI,rankCWUR]).groupby('nome_università',as_index=False).count()

,nome_università,world_rank
0,AGH University of Science and Technology,2
1,Aalborg University,3
2,Aalto University,3
3,Aarhus University,3
4,Aberystwyth University,2
...,...,...
1447,École centrale de Lyon,1
1448,École normale supérieure - Paris,1
1449,École normale supérieure de Cachan,1
1450,École normale supérieure de Lyon,1


# PUNTO 11: In the times ranking, compute the number of times each university appears

In [31]:
times_data.groupby('university_name',as_index=False).size()

,university_name,size
0,AGH University of Science and Technology,1
1,Aalborg University,5
2,Aalto University,5
3,Aarhus University,6
4,Aberystwyth University,5
...,...,...
813,École Normale Supérieure,6
814,École Normale Supérieure de Lyon,6
815,École Polytechnique,6
816,École Polytechnique Fédérale de Lausanne,6


# PUNTO 12: Find the universities that appear at most twice in the times ranking

In [32]:
numero_app=times_data.groupby('university_name',as_index=False).size()
numero_app[numero_app['size']<=2]

,university_name,size
0,AGH University of Science and Technology,1
5,Adam Mickiewicz University,1
6,Aix-Marseille University,1
7,Ajou University,1
9,Alexandru Ioan Cuza University,1
...,...,...
806,Yokohama City University,1
807,Yokohama National University,1
810,Yuan Ze University,2
811,Yıldız Technical University,1


# PUNTO 13: The universities that, in any year, have the same position in all three rankings (they must have the same position in a year)

In [33]:
FUSO=pd.merge(shanghaiData,times_data,on='nome_università')
FUSO1=pd.merge(cwurData,FUSO,on='nome_università')
FUSO1.head()[['nome_università','world_rank','world_rank_x','world_rank_y']]
FUSO1['world_rank']=np.array(FUSO1['world_rank'],dtype=str)
FUSO1[['nome_università','world_rank','world_rank_x','world_rank_y']].head()

,nome_università,world_rank,world_rank_x,world_rank_y
0,Harvard University,1,1,1
1,Harvard University,1,1,2
2,Harvard University,1,1,4
3,Harvard University,1,1,2
4,Harvard University,1,1,2


In [34]:
FUSO2=FUSO1[(FUSO1['world_rank']==FUSO1['world_rank_x'])&(FUSO1['world_rank_x']==FUSO1['world_rank_y'])]
FUSO2[['nome_università','world_rank','world_rank_x','world_rank_y']].tail()

,nome_università,world_rank,world_rank_x,world_rank_y
2473,University of Chicago,9,9,9
2475,University of Chicago,9,9,9
2479,University of Chicago,9,9,9
2481,University of Chicago,9,9,9
3164,Johns Hopkins University,16,16,16


In [35]:
FUSO2.groupby('nome_università',as_index=False).count()['nome_università']

0          Harvard University
1    Johns Hopkins University
2        Princeton University
3         Stanford University
4     University of Cambridge
5       University of Chicago
6             Yale University
Name: nome_università, dtype: object